In [15]:
# Import necessary modules
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.serializer import GraphSONSerializersV3d0
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P, T, Direction

import nest_asyncio

# Apply nest_asyncio to allow nested event loops (useful in Jupyter notebooks)
nest_asyncio.apply()

# Create a GraphSON v3 serializer to ensure compatibility with the Gremlin server
serializer = GraphSONSerializersV3d0()

# Connect to the Gremlin server using the serializer
graph = Graph()
connection = DriverRemoteConnection("ws://localhost:8182/gremlin", "g", message_serializer=serializer)
g = graph.traversal().withRemote(connection)


# Helper function to ensure all properties are simple data types
def sanitize_properties(properties):
    for key, value in properties.items():
        if not isinstance(value, (str, int, float, bool)):
            raise ValueError(f"Property '{key}' has an unsupported type: {type(value)}. Only primitive types are allowed.")
    return properties


# Function to get or create a vertex with unique properties, no updating
def get_or_create_vertex(label, properties):
    properties = sanitize_properties(properties)  # Ensure properties are valid

    unique_key = "name"
    unique_value = properties.get(unique_key)
    if unique_value is None:
        raise ValueError("Unique property 'name' is required to create or get a vertex.")
    
    # Attempt to find an existing vertex with the given label and unique property
    vertex_traversal = g.V().hasLabel(label).has(unique_key, unique_value)
    if vertex_traversal.hasNext():
        return vertex_traversal.next()
    else:
        # Create a new vertex with the provided properties
        vertex = g.addV(label)
        for key, value in properties.items():
            vertex = vertex.property(key, value)
        return vertex.next()


# Function to get or create an edge between two vertices
def get_or_create_edge(from_vertex, to_vertex, label, properties):
    properties = sanitize_properties(properties)  # Ensure properties are valid

    # Attempt to find an existing edge with the given label between the two vertices
    edge_traversal = g.V(from_vertex.id).outE(label).where(__.inV().hasId(to_vertex.id))

    if edge_traversal.hasNext():
        return edge_traversal.next()
    else:
        # Create a new edge with the provided properties
        edge = g.V(from_vertex.id).addE(label).to(__.V(to_vertex.id))
        for key, value in properties.items():
            edge = edge.property(key, value)
        return edge.next()


# Function to retrieve all vertices within N hops from a starting vertex
def get_vertices_within_hops(start_vertex_id, hops):
    vertices_within_hops = (
        g.V(start_vertex_id)
        .emit()  # Emit the starting vertex
        .repeat(__.both().simplePath())
        .times(hops)
        .dedup()
        .elementMap()
        .toList()
    )
    return vertices_within_hops


# Function to retrieve edges between a list of vertex IDs
def get_edges_between_vertices(vertex_ids):
    edges_between_vertices = (
        g.V(vertex_ids)
        .bothE()
        .where(__.otherV().hasId(P.within(vertex_ids)))
        .dedup()
        .elementMap()
        .toList()
    )
    return edges_between_vertices


# Drop all vertices (and connected edges) to start fresh
g.V().drop().iterate()

# Create or get vertices
taewoon = get_or_create_vertex("Person", {"name": "Taewoon"})
eert = get_or_create_vertex("Person", {"name": "Eert"})
amsterdam = get_or_create_vertex("City", {"name": "Amsterdam"})
netherlands = get_or_create_vertex("Country", {"name": "Netherlands"})
europe = get_or_create_vertex("Continent", {"name": "Europe"})
asia = get_or_create_vertex("Continent", {"name": "Asia"})

# Create or get edges
lives_in = get_or_create_edge(taewoon, amsterdam, "lives_in", {"current_time": "2024-10-27"})
lives_in = get_or_create_edge(eert, amsterdam, "lives_in", {"current_time": "1993-10-27"})
capital_of = get_or_create_edge(amsterdam, netherlands, "capital_of", {})
include = get_or_create_edge(europe, netherlands, "include", {})
next_to = get_or_create_edge(europe, asia, "next_to", {})
# visit = get_or_create_edge(taewoon, netherlands, "visit", {"date": "2024-10-27"})


# Specify the trigger node and number of hops
trigger_node_label = "Person"
trigger_node_name = "Taewoon"
hops = 2

# Find the starting vertex
start_vertex_traversal = g.V().hasLabel(trigger_node_label).has("name", trigger_node_name)
if not start_vertex_traversal.hasNext():
    raise ValueError(f"Vertex with label '{trigger_node_label}' and name '{trigger_node_name}' not found.")
start_vertex = start_vertex_traversal.next()

# Get all vertices within N hops from the starting vertex
vertices_within_hops = get_vertices_within_hops(start_vertex.id, hops)

# Collect the IDs of the vertices using T.id
vertex_ids = [v[T.id] for v in vertices_within_hops]

# Get all edges between the collected vertices
edges_between_vertices = get_edges_between_vertices(vertex_ids)

# Print the vertices within hops
print(f"Vertices within {hops} hops from '{trigger_node_name}':")
for v in vertices_within_hops:
    # Extract properties excluding T.id and T.label
    properties = {k: v[k] for k in v if k not in (T.id, T.label)}
    print({"id": v[T.id], "label": v[T.label], "properties": properties})

# Print the edges between vertices within hops
print("\nEdges between vertices within hops:")
for e in edges_between_vertices:
    out_vertex = e[Direction.OUT]
    in_vertex = e[Direction.IN]
    # Extract edge properties excluding T.id, T.label, Direction.OUT, Direction.IN
    properties = {
        k: e[k] for k in e if k not in (T.id, T.label, Direction.OUT, Direction.IN)
    }
    print(
        {
            "id": e[T.id],
            "label": e[T.label],
            "outV": {"id": out_vertex[T.id], "label": out_vertex[T.label]},
            "inV": {"id": in_vertex[T.id], "label": in_vertex[T.label]},
            "properties": properties,
        }
    )

# Optional: Close the connection when done
connection.close()


Vertices within 2 hops from 'Taewoon':
{'id': 233648, 'label': 'Person', 'properties': {'name': 'Taewoon'}}
{'id': 291016, 'label': 'City', 'properties': {'name': 'Amsterdam'}}
{'id': 286928, 'label': 'Country', 'properties': {'name': 'Netherlands'}}
{'id': 286920, 'label': 'Person', 'properties': {'name': 'Eert'}}

Edges between vertices within hops:
{'id': {'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': '2m1i-50a8-i6t-68js'}}, 'label': 'lives_in', 'outV': {'id': 233648, 'label': 'Person'}, 'inV': {'id': 291016, 'label': 'City'}, 'properties': {'current_time': '2024-10-27'}}
{'id': {'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': '33fd-68js-bv9-65e8'}}, 'label': 'capital_of', 'outV': {'id': 291016, 'label': 'City'}, 'inV': {'id': 286928, 'label': 'Country'}, 'properties': {}}
{'id': {'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': '3315-65e0-i6t-68js'}}, 'label': 'lives_in', 'outV': {'id': 286920, 'label': 'Person'}, 'inV': {'i